# Setting up a new deep learning station with Python 3.6, Keras, Tensorflow and OpenCV using Ubuntu 16.04 on a Google Cloud VM instance

## Step 1: Create a new VM instance with a GPU

### 0) Generate your own SSH key pair which you will use to connect to your instance after creating it.

You may skip this step if you already have an SSH key, or if you are sure you will log into your instance only from Google Cloud Console. 

Otherwise, create a new SSH key (a public/private key pair) on your computer. The specific procedure for doing this depends on your system. In most environments, you can use ssh-keygen in the command line. In Windows, use PuttyGen software (download and install Putty from https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html). 

If you are not sure how to create an SSH key, there are many tutorials online.

Save your SSH key in a file.


### 1) Log in to Google Cloud Console


### 2) Go to Menu (left upper corner) -> IAM and admin -> Quotas

Select Metric -> NVIDIA K80 GPUs

Among available locations, choose the one closest to you geographically

If the current GPU quota for your chosen location is 0, click Edit Quotas and set the quota limit to 1

In Request description, explain what you will need the GPU for.

If you are new to Google Cloud, your request will probably not be approved automatically. You may need to make a deposit and contact support in order to be able to use a GPU.


### 3) Go to Menu (left upper corner) -> Compute Engine -> VM instances

Click the blue Create button.


Specify the instance parameters (these would depend on your budget and goals). 
Here are my choices:

<i>Zone</i>: 
    
    Same zone for which you have the GPU quota approved.

<i>Machine type</i>: 
    
    Click Customize, choose Cores: 4 (4 vCPU), Memory: 26 GB, CPU Platform: Inter Broadwell or later, GPUs: 1 NVIDIA Tesla K80.

<i>Container</i>:
    
    Leave unchecked.

<i>Boot disk</i>: 

    OS Images: Ubuntu 16.04 LTS
    
    Boot Disk Type: SSD persistent disk
    
    Size: 100GB

<i>Identity and API access</i>:
    
    Keep default options
    
<i>Firewall</i>:
    
    Select both Allow HTTP traffic and Allow HTTPS traffic
    
<i>Management, disks, networking, SSH keys</i>:
    
    Do not change anything. I am not sure if there is a bug in Google Cloud Console or I just don't understand how this is supposed to work, but whatever SSH keys I tried setting either here or globally for the project did not get saved on the VM instance. Instead, Cloud Console saved its own SSH keys, so until I realized what was happening, I was not able to access the VM instance using non-console SSH/SCP/FTP tools. 
    
After making all of these selections, click Create and confirm that you are willing to be billed for this instance.

### 4) Once the instance has been created, click on the SSH button in the row corresponding to this instance under Connect header

A new window will open, and after about a minute of authentication you will be able to access the Ubuntu command line tools on your new VM instance

### 5) Add your own SSH key to the instance so that you can log in using tools other than Google Cloud Console

I am using Vi text file editor to perform this step. If you have a different favorite editor, feel free to use it instead. What you need to do is add the public SSH key you generated in part 0 above to the ~/.ssh/authorized_keys file.

In the SSH window for your new instance, type 
> sudo vi ~/.ssh/authorized_keys

Scroll to the end of the file.
Press "a" key to enable editing.
Append your generated public key to the end of the file.
Press the "ESC" key and then type 
> :wq 

followed by the "Enter" key. This will save the changes on disk.

Close the SSH window and verify that you can log into your VM instance using both the Cloud Console and other software that you might have. In the latter case you will need to provide your software with the SSH key that you saved on the VM instance.

When editing important files by hand as we did above, it is possible to make a mistake that would render the file useless, in which case you might not be able to log in. If this happens, delete your newly created VM instance in the Google Cloud Console (click on the triple dot at the end of the row corresponding to your instance and choose Delete) and try again starting from step 3 above. 

## Step 2: Install CUDA 9.0

This part of the instructions is heavily based on https://yangcha.github.io/CUDA90/


### 1) Create a directory for downloads to keep things neat

> mkdir downloads

> cd downloads

### 2) Make sure the system is updated

> sudo apt-get update

> sudo apt-get upgrade

### 3) Install unzip

> sudo apt-get install unzip

### 4) Download CUDA 9.0 files

> sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub

> wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

> wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.0.5.15-1+cuda9.0_amd64.deb

> wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7-dev_7.0.5.15-1+cuda9.0_amd64.deb

> wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libnccl2_2.1.4-1+cuda9.0_amd64.deb

> wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libnccl-dev_2.1.4-1+cuda9.0_amd64.deb

### 5) Unpack the downloaded files

> sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

> sudo dpkg -i libcudnn7_7.0.5.15-1+cuda9.0_amd64.deb

> sudo dpkg -i libcudnn7-dev_7.0.5.15-1+cuda9.0_amd64.deb

> sudo dpkg -i libnccl2_2.1.4-1+cuda9.0_amd64.deb

> sudo dpkg -i libnccl-dev_2.1.4-1+cuda9.0_amd64.deb

> sudo apt-get update

### 6) Install the downloaded files (takes about 5 minutes)

> sudo apt-get install cuda=9.0.176-1


### 7) Reboot the system to load the NVIDIA drivers

> sudo reboot

At this point you should get disconnected from your SSH session. Give your VM instance a minute to reboot, then restart the session.

### 8) Adjust the paths to include CUDA locations

Open the .bashrc file for editing using Vi.

> sudo vi ~/.bashrc

Add the following two lines to the end of the file (as in Step 1, part 5 above where we added an SSH key):

Reload the .bashrc file

> source ~/.bashrc

### 9) Verify that CUDA has been installed

Type

> nvcc --version

You should get CUDA 9.0 as your current version. Here's the full output:

### The following seven steps are largely based on https://www.pyimagesearch.com/2016/10/24/ubuntu-16-04-how-to-install-opencv/ 

## Step 3: Install build tools

> cd downloads

> sudo apt-get update

> sudo apt-get upgrade

> sudo apt-get install build-essential cmake pkg-config

> sudo apt-get install libjpeg8-dev libtiff5-dev libjasper-dev libpng12-dev

> sudo apt-get install libavcodec-dev libavformat-dev libswscale-dev libv4l-dev

> sudo apt-get install libxvidcore-dev libx264-dev

> sudo apt-get install libgtk-3-dev

> sudo apt-get install libatlas-base-dev gfortran

## Step 4: Install Python 3.6

> sudo add-apt-repository ppa:deadsnakes/ppa
    
> sudo apt-get update

> sudo apt-get install python2.7-dev python3.6-dev

## Step 5: Download OpenCV 3.4.1

> wget -O opencv.zip https://github.com/Itseez/opencv/archive/3.4.1.zip

> unzip opencv.zip

> wget -O opencv_contrib.zip https://github.com/Itseez/opencv_contrib/archive/3.4.1.zip

> unzip opencv_contrib.zip

## Step 6: Create a virtual environment for Python 3.6

Install virtual environments

> wget https://bootstrap.pypa.io/get-pip.py

> sudo python get-pip.py

> sudo pip install virtualenv virtualenvwrapper

> sudo rm -rf get-pip.py .cache/pip

Open the .bashrc file for editing using Vi.

> sudo vi ~/.bashrc

Add the two lines below to ~/.bashrc 

> export WORKON_HOME=$HOME/.virtualenvs

> source /usr/local/bin/virtualenvwrapper.sh

Reload .bashrc

> source ~/.bashrc

Create a new virtual environment named p36

> mkvirtualenv p36 -p python3.6

## Step 7: Install Numpy and compile OpenCV from source (the last command takes a long time to finish, about an hour and 20 minutes)

> pip install numpy

> cd opencv-3.4.1/

> mkdir build

> cd build

> cmake -D CMAKE_BUILD_TYPE=RELEASE \
    -D CMAKE_INSTALL_PREFIX=/usr/local \
    -D INSTALL_PYTHON_EXAMPLES=ON \
    -D INSTALL_C_EXAMPLES=OFF \
    -D OPENCV_EXTRA_MODULES_PATH=~/downloads/opencv_contrib-3.4.1/modules \
    -D PYTHON_EXECUTABLE=~/.virtualenvs/p36/bin/python \
    -D BUILD_EXAMPLES=ON ..

> make -j4

## Step 8: Manually add Python 3.6 bindings

This step is essential to get the installation to finish successfully. It is based on https://github.com/opencv/opencv/issues/10771

> deactivate

> python ../modules/python/src2/gen2.py ../build/modules/python_bindings_generator ../build/modules/python_bindings_generator/headers.txt

## Step 9: Install OpenCV and adjust file names

> sudo make install

> sudo ldconfig

> cd /usr/local/lib/python3.6/site-packages/

> sudo mv cv2.cpython-36m-x86_64-linux-gnu.so cv2.so

> cd ~/.virtualenvs/p36/lib/python3.6/site-packages/

> ln -s /usr/local/lib/python3.6/site-packages/cv2.so cv2.so

## Step 10: Install other machine learning packages

> workon p36

Below are the packages I installed. Obviously choose the ones that you need.

> pip install pandas

> pip install scikit-learn

> pip install scikit-image

> pip install h5py

> pip install catboost

> pip install tensorflow-gpu

> pip install keras

> pip install dlib

> pip install http://download.pytorch.org/whl/cu90/torch-0.3.1-cp36-cp36m-linux_x86_64.whl

> pip install torchvision

## And we are done!